In [25]:
#age in code ro nafahmidin khodetoon ye code joda ba estefade az StreamData_preprocessed.csv benevisin. yani
#ye code benevisin ke StreamData_preprocessed.csv ro load kone va satr hasho yeki yeki pass bede be codetoon.
#input: 7-column-rows
#output: send the rows to elasticsearch and show it.
import json,datetime
from kafka import KafkaConsumer
import pandas as pd
from elasticsearch import Elasticsearch

ElasticsearchPort = 9200
elasticIndex = 'bigdatafinalproject3'
First_Time = True
pathToStreamData = r'C:\Users\AhmaDGoly\Desktop\Arshad\TERM 2\Big Data\Final\Codes\StreamData_preprocessed.csv'

In [3]:
def current_time():
    current_datetime = datetime.datetime.now()
    return current_datetime.strftime("[%Y-%m-%d_%H:%M:%S]")

In [7]:
es = Elasticsearch(hosts=['http://localhost:9200'])
print(f'{current_time()}: Connected to elasticsearch')

[2023-07-14_17:15:00]: Connected to elasticsearch


In [ ]:
if First_Time:
    index_name = elasticIndex
    index_body = {
        'settings': {
            'number_of_shards': 1,
            'number_of_replicas': 0
        },
        'mappings': {
            'properties': {
                    "date": {
                        "type": "date"
                    },
                    "message": {
                        "type": "text"
                    },
                    "hashtags": {
                        "type": "keyword"
                    },
                    "score": {
                        "type": "integer"
                    },
                    "category": {
                        "type": "keyword"
                    },
                    "clean_message": {
                        "type": "text"
                    },
                    "ML_score": {
                        "type": "float"
                    }
                
            }
        }
    }
    es.indices.create(index=index_name, body=index_body)
    print(f"{current_time()}: First_time is on. created the index called {elasticIndex}")
    First_Time = False


In [19]:
import dateutil.parser
df = pd.read_csv(pathOldData, encoding='utf-8-sig')


In [ ]:
x = 0
for index, row in df.iterrows():
    x = x + 1
    print(f"{current_time()}: {x}: Received a news from Preprocessing_Thread...")
    receivedRowDF = pd.DataFrame([row])
    receivedRowDF['date'] = receivedRowDF['date'].apply(lambda x: dateutil.parser.parse(x).isoformat())
    records = receivedRowDF.to_dict(orient='records')
    for record in records:
        res = es.index(index=elasticIndex, document=record)
    print(f"{current_time()}: {x}: Saved on Elasticsearch on index = {elasticIndex}")